This version does away with diagonal movement: it is probably superfluous. Also added Box counting dimension

In [ ]:
#Function creates a Diffusion Limited Aggregation Cluster from a 2-D base seed array with the specified number of cells.
#In the baseseed array, -1's indicate empty sites, 0's indicate empty and origin sites, 1's indicate the seed, 2's are other
#boundaries if needed. The baseseed array must be a square.
#Returns a 2x1 array with the cluster in the first index and the age tracker in the second index
#Sticky parameter controls chance of particle not sticking. 0 will always stick, 1 will never stick (don't pick 1)
def DLACluster(baseseed, numcells, sticky):

    import numpy as np
    import numpy.random as nprand
    import matplotlib.pyplot as plt

    body = baseseed
    ncells = numcells
    nside = len(baseseed[0])

    #create an array of neighbor coordinates
    neighbors=np.array([
        #adjacent neighbors
        (0, 1), 
        (0, -1), 
        (1, 0), 
        (-1, 0),
        #diagonal neighbors
        (1, 1),
        (1, -1),
        (-1, -1),
        (-1, 1),
    ])

    age=np.zeros((nside,nside))    #make an array that holds the step number for which the cell is occupied

    cellcounter = 1     #track the number of cells, starts at 1 with the seed

    while cellcounter < ncells:         #add the new sites until the target number is reached
        #TO-DO: Set body points to 0 within square of length cellcounter*2+5 to cut down on run-time
        #if cellcounter*2+5 < nside:
        #    y = x = ctr - cellcounter - 2 #start at top left hand corner of square
         #   for i in np.arange(cellcounter*2+5):
          #      y += 1
           #     x = ctr - cellcounter - 2
            #    for j in np.arange(cellcounter*2+5):
             #       x += 1
              #      if body[y,x] == -1:
               #         body[y,x] = 0     #set -1's to 0's to make them candidates

        #Make a random walker at a random point
        gsites = np.where(body==0)     #find indices of available sites, (empty indicies within square)
        nget=np.random.choice(gsites[0].size)  #randomly select the gsite index
        ypos=gsites[0][nget]    #cumulative y position, starts at randomly selected point
        xpos=gsites[1][nget]    #cumulative x position, starts at randomly selected point

        friend = False  #tracks whether a neighbor has been found
        stepcount = 0   #counts number of steps taken

        for near in neighbors: #check if the walker started next to a neighbor
            if xpos < nside-1 and xpos > 0 and ypos < nside-1 and ypos > 0: #makes sure
                if body[ypos+near[0], xpos+near[1]] == 1:
                    friend = True

        while friend == False: # and stepcount <= len(body)*5: #make steps until a neighbor is found, abort after too many steps     
            rand = nprand.rand() #make a random number from 0 to 1
            x = 0
            y = 0
            barrier = True #tracks whether the current step will hit the edge of the body or cross another cell

            if xpos == nside-1:
                xpos += -1
            if ypos == nside-1:
                ypos += -1
            if xpos == 0:
                xpos += 1
            if ypos == 0:
                ypos += 1

            #Make a step, also checks for barriers
            if rand < .250 and rand >= 0.00: #walk N
                x = 0
                y = -1  
            if rand < .500 and rand >= .250: #walk E
                x = 1
                y = 0
            if rand < .750 and rand >= .500: #walk S
                x = 0
                y = 1  
            if rand < 1.00 and rand >= .750: #walk W
                x = -1
                y = 0

            #Checks for a barrier
            if body[ypos+y, xpos+x] != (1 and 2): #checks if there is a barrier at the desired step direction
                barrier = False

            #print(xpos, ypos)
            #print(x, y)
            #print(barrier)
            #print(body[ypos+y, xpos+x])
            #print(rand)
            #print()

            if barrier == False: #increment positions and stepcount if there isn't a barrier
                xpos += x
                ypos += y
                stepcount += 1 

            if barrier == True: #move in opposite direction if there is a barrier
                xpos += -x
                ypos += -y
                stepcount += 1

            #Checks if there is a neighbor, a horizontally or vertically adjacent cell
            for near in neighbors:
                if xpos < nside-1 and xpos > 0 and ypos < nside-1 and ypos > 0:
                    #print(xpos, ypos)
                    if body[ypos+near[0], xpos+near[1]] == 1:
                        friend = True

            #Adds the walker if there is a neighbor and the particle sticks
            if friend == True and nprand.rand() >= sticky:
                body[ypos,xpos] = 1    #Add the final position of the walker to the cluster
                cellcounter += 1      #Increment the number of cells
                age[ypos,xpos]= ncells-cellcounter   #record the age for the filled site at the end of the run
            else:
                friend = False

    return [body, age]

    #Function for finding the box counting dimension of a cluster.

import math as m

# From https://github.com/rougier/numpy-100 (#87)

def boxcount(inarr, k): #inarr is the array of cell positions and k is the box size
    S = np.add.reduceat(
        np.add.reduceat(inarr, np.arange(0, inarr.shape[0], k), axis=0),
                            np.arange(0, inarr.shape[1], k), axis=1)

    # We count non-empty (0) and non-full boxes (k*k)
    return len(np.where((S > 0))[0]) #output is the number of boxes with stuff in them

#Get the number of boxes as a function of box size then take the logarithm of both values.
#All -1's and 2's in the body are converted to 0's
def boxdimension(body, ncells):
    
    #Convert -1's and 2's to 0's
    for i in range(len(body)):
        for j in range(len(body)):
            if body[i, j] == (-1 or 2):
                body[i, j] = 0
    
    boxsizes = [1]
    boxnum = [ncells]
    logsize = []
    lognum = []
    
    size = 2
    while boxnum[-1] > 1:
        boxsizes.append(size)
        boxnum.append(boxcount(body, size))
        size += 1
        
    #Takes the logarithm of both sides (default is natural log)
    #We cut off later points because the data loses a consistent trend
    for i in np.arange(int(len(boxsizes)/2)):
        logsize.append(m.log(boxsizes[i]))

    for i in np.arange(int(len(boxnum)/2)):
        lognum.append(m.log(boxnum[i]))
        
    dimension = -1*np.polyfit(logsize, lognum, 1)
    
    return dimension[0]

In [ ]:
#Models the electrodeposition of copper sulfate.
#Seed: center point seed
#Walkers: all boundaries
#Stickiness: medium

import numpy as np
import numpy.random as nprand
import matplotlib.pyplot as plt

nside1=50   #width of the grid, add an extra two to the width to be made into boundaries
ncells1=100  #specify number of sites to fill
sticky1=0.7 #sets stickiness of walkers

body1=np.zeros((nside1,nside1))-1#array holding the cell occupancy counter (-1 = empty; 0 = candidate and empty; 1 = filled; 
# 2 = edge)

#Set Origin sites around boundary
body1[-1,:] = 0   #bottom
body1[0,:] = 0   #top
body1[:,0] = 0   #left
body1[:,-1] = 0   #right

ctr=int(nside1/2)    #find the center
body1[ctr,ctr] = 1   #seed the initial point at the center

cluster1 = DLACluster(body1, ncells1, sticky1) #create the cluster

dim1 = boxdimension(cluster1[0], ncells1) #find the fractal dimension

print("side, cell, sticky, fractal")
print(nside1, ncells1, sticky1, dim1)

plt.figure(figsize=(15,15))
plt.imshow(cluster1[0],cmap=plt.cm.hot_r)
#plt.title("Point Cluster")
plt.xlabel("X position")
plt.ylabel("Y position")
plt.savefig("point cluster.png")
plt.show()    

In [ ]:
plt.figure(figsize=(15,15))
plt.imshow(cluster1[1],cmap=plt.cm.hot_r)
#plt.title("Point Cluster")
plt.xlabel("X position")
plt.ylabel("Y position")
plt.savefig("point cluster.png")
plt.show()  

In [ ]:
#Models the growth of Manganese crystals on a plate.
#Seed: line seed on bottom
#Walkers: top boundary
#Stickiness: high

import numpy as np
import numpy.random as nprand
import matplotlib.pyplot as plt

nside2=200   #width of the grid, add an extra two to the width to be made into boundaries
ncells2=6000 #specify number of sites to fill
sticky2= .90

body2=np.zeros((nside2,nside2))-1  #array holding the cell occupancy counter (-1 = empty; 0 = candidate and empty; 1 = filled; 
# 2 = edge)

#Set Origin sites on the top
body2[0,:] = 0 #top

body2[-1,:] = 1   #seed the floor

cluster2 = DLACluster(body2, ncells2, sticky2) #create the cluster

dim2 = boxdimension(cluster2[0], ncells2) #find the fractal dimension

print("side, cell, sticky, fractal")
print(nside2, ncells2, sticky2, dim2)

plt.figure(figsize=(15,15))
plt.imshow(cluster2[1],cmap=plt.cm.hot_r)
#plt.title("Wall Cluster")
plt.xlabel("X position")
plt.ylabel("Y position")
plt.savefig("wall cluster.png")
plt.show()    

In [ ]:
plt.figure(figsize=(15,15))
plt.imshow(cluster2[0],cmap=plt.cm.hot_r)
#plt.title("Point Cluster")
plt.xlabel("X position")
plt.ylabel("Y position")
plt.savefig("wall cluster.png")
plt.show() 

In [ ]:
#Models the growth of coral on the seafloor.
#Seed: point seed on bottom boundary
#Walkers: left, top, and right boundary (WIP)
#Stickiness: medium

import numpy as np
import numpy.random as nprand
import matplotlib.pyplot as plt

nside3=200   #width of the grid, add an extra two to the width to be made into boundaries
ncells3=4000 #specify number of sites to fill
sticky3=0.7

body3=np.zeros((nside3,nside3))-1  #array holding the cell occupancy counter (-1 = empty; 0 = candidate and empty; 1 = filled; 
# 2 = edge)

#Set Origin sites around left, top, and right
body3[0,:] = 0   #top
body3[:,0] = 0   #left
body3[:,-1] = 0   #right

ctr=int(nside3/2)    #find the center
body3[-1,ctr] = 1   #seed the initial point at the center of the bottom

cluster3 = DLACluster(body3, ncells3, sticky3) #create the cluster

dim3 = boxdimension(cluster3[0], ncells3) #find fractal dimension

print("side, cell, sticky, fractal")
print(nside3, ncells3, sticky3, dim3)

plt.figure(figsize=(15,15))
plt.imshow(cluster3[1],cmap=plt.cm.hot_r)
#plt.title("Point-Wall Cluster")
plt.xlabel("X position")
plt.ylabel("Y position")
plt.savefig("point wall cluster.png")
plt.show() 

In [ ]:
plt.figure(figsize=(15,15))
plt.imshow(cluster3[0],cmap=plt.cm.hot_r)
#plt.title("Point Cluster")
plt.xlabel("X position")
plt.ylabel("Y position")
plt.savefig("coral cluster.png")
plt.show() 

In [ ]:
#Models the growth of frost crystals on a window. 
#Seed: boundary
#Walkers: all open cells
#Stickiness: low

import numpy as np
import numpy.random as nprand
import matplotlib.pyplot as plt

nside4=200   #width of the grid, add an extra two to the width to be made into boundaries
ncells4=28000 #specify number of sites to fill
sticky4=.99

body4=np.zeros((nside4,nside4))#array holding the cell occupancy counter (-1 = empty; 0 = candidate and empty; 1 = filled; 
# 2 = edge)

#Set Origin sites at the center
ctr=int(nside4/2)    #find the center
body4[ctr,ctr] = 0   #origin at the center

#seed the boundary
body4[-1,:] = 1   #seed the bottom
body4[0,:] = 1   #seed the top
body4[:,0] = 1   #seed the left
body4[:,-1] = 1   #seed the right

cluster4 = DLACluster(body4, ncells4, sticky4) #create the cluster

dim4 = boxdimension(cluster4[0], ncells4) #find fractal dimension

print("side, cell, sticky, fractal")
print(nside4, ncells4, sticky4, dim4)

plt.figure(figsize=(15,15))
plt.imshow(cluster4[1],cmap=plt.cm.hot_r)
#plt.title("Box Cluster")
plt.xlabel("X position")
plt.ylabel("Y position")
plt.savefig("box cluster.png")
plt.show()    

In [ ]:
plt.figure(figsize=(15,15))
plt.imshow(cluster4[0],cmap=plt.cm.hot_r)
#plt.title("Point Cluster")
plt.xlabel("X position")
plt.ylabel("Y position")
plt.savefig("frost cluster.png")
plt.show() 